# Geração do dataset
O dataset gerado por este scrip está separado em treino/teste e organizado por classes, pronto para ser utilizado como generator do Tensorflow.
Foi criado uma classe Dataset, de onde cada dataset herda uma classe específica. Ex.: o dataset "covid-chestxray-dataset (<https://github.com/ieee8023/covid-chestxray-dataset>)" é instanciado pela classe *cohen()*.

O pipeline de processamento é:<br>
1 - import das bibliotecas<br>
2 - instanciação dos objetos de cada dataset<br>
3 - Rotina de leitura de cada dataset, aplicando as devidas funções<br>
Obs.: nem todos os datasets possuem funções de prefiltragem/posfiltragem. Essas funções foram definidas conforme o processamento feito em <https://github.com/lindawangg/COVID-Net><br>
4 - Junção das tabelas de cada ds em uma só (com excessão do RSNA)<br>
5.1a - Filtragem das classes de interesse na tabela de imagens<br>
5.1b - Separação de imagens específicas para teste, conforme <https://github.com/lindawangg/COVID-Net><br>
5.2a - Filtragem das classes de interesse na tabela do RSNA<br>
5.2b - Aplica *split* no dataset RSNA<br>
6 - Junta as tabelas de treino e teste<br>
7 - Monta o dataset no path destino, copiando as imagens que já estão em formato de leitura e escrevendo as imagens em ```.dcm```<br>
<br>
Obs.: as estapas 3 e 7 podem demorar consideravelmente devido ao dataset RSNA que possui mais de 15k imagens. A fim de verificar a validade do script recomenda-se rodá-lo sem este dataset.<br>
<br>
As tables ou tabelas referidas nesse script são listas de dicionário do tipo \[{"path": target_path, "filename": filename, "class": finding, "url": url, "id": patientid}\], onde:<br>
- target_path = localização da imagem dentro do diretorio do dataset;<br>
- filename = nome do arquivo, presente no dataset;<br>
- class = classe de classificação da imagem. Ex.: Normal, COVID-19, etc;<br>
- url = URL da imagem, usado para detecção de imagens presentes simultaneamente em dois ou mais datasets. Se não há "url" no dataset, o valor None é preenchido na tabela;<br>
- patientid = utilizado para busca rapida de alguns paciente. Se não há "patientid" no dataset, o nome do arquivo sem extensão é utilizado.<br>



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# installl requirements on google colab
!pip install -r drive/MyDrive/TG/tg2_COVIDNet/requirements.txt


     |████████████████████████████████| 133kB 13.8MB/s 
     |████████████████████████████████| 174kB 26.6MB/s 
     |████████████████████████████████| 163kB 54.8MB/s 
     |████████████████████████████████| 829kB 60.6MB/s 
     |████████████████████████████████| 122kB 52.3MB/s 
     |████████████████████████████████| 4.0MB 57.7MB/s 
     |████████████████████████████████| 122kB 65.0MB/s 
     |████████████████████████████████| 788kB 53.2MB/s 
     |████████████████████████████████| 122kB 52.8MB/s 
     |████████████████████████████████| 1.4MB 62.7MB/s 
     |████████████████████████████████| 307kB 59.7MB/s 
     |████████████████████████████████| 61kB 11.0MB/s 
     |████████████████████████████████| 112kB 66.1MB/s 
     |████████████████████████████████| 71kB 12.1MB/s 
     |████████████████████████████████| 552kB 47.7MB/s 
     |████████████████████████████████| 9.5MB 46.2MB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 49.5MB 60k

## IMPORT DA LIBS E FUNÇÕES

In [ ]:
import numpy as np
import pandas as pd
import os
import random 
import pydicom as dicom
import cv2

In [ ]:
os.chdir('drive/MyDrive/TG/tg2_COVIDNet')


In [ ]:
from datasets import rsna, actualmed, cohen, fig1, sirm
from ds_utils import filter_table, split_table, table_info, remove_dupl_field

## CONSTROI OS OBJ DE CADA DS

In [ ]:
print(os.getcwd())

In [ ]:
# datasets_path = os.path.join(os.getcwd(), "datasets")
datasets_path = "/Users/igorbeduin/Google Drive (beduinigor@gmail.com)/TG/datasets"

cohen = cohen(datasets_path)
rsna = rsna(datasets_path)
actualmed = actualmed(datasets_path)
fig1 = fig1(datasets_path)
sirm = sirm(datasets_path)

## LE CADA DS E CONSTROI SUA TABELA APLICANDO AS FUNCOES NECESSARIAS

In [ ]:
datasets = [cohen, actualmed, fig1, sirm, rsna]
for ds in datasets:
    ds.read()
    ds.prefilter()
    ds.mount_table()
    ds.postfilter()
    ds.mount_count_table()
    if ds.__name__ is sirm.__name__:
        remove_dupl_field(sirm, cohen, "url")

## JUNTA AS TABELAS EM UMA SÓ

In [ ]:
target_ds = [cohen, actualmed, fig1, sirm]
file_table = []
for ds in target_ds:
    print(f"Dataset: {ds.__name__.upper()}")
    print("-----------------------------------------")
    print(f"Imagens: {ds.count}")
    print(f"Contagem de cada classe por dataset: {ds.count_table}\n")
    file_table += ds.table

print(f"Total de imagens: {len(file_table)}")

## FILTRA A TABELA PARA USO APENAS DAS CLASSES DE INTERESSE
Se general_case="remove":<br>
    - Se a classe da imagen não está em mapping, a imagem é removida da table<br>
Se general_case="subst":<br>
    - Se a classe da imagen não está em mapping, a classe da imagem é modificada para o valor em "std_subst"<br>
        Para este caso é possível passar uma lista ```remove_classes```, assim a classe não presente em mapping será substituída no caso padrão mas será removida da table se estiver presente na lista. <br>
        Ex.:<br>
        - class = "blabla" será atribuída o valor de mapping["std_subst"]<br>
        - class = "todo" terá a imagem removida da tabela<br>

In [ ]:
mapping = {"COVID-19": "COVID-19",
           "COVID-19, ARDS": "COVID-19",
           "Normal": "Normal",
           "Pneumonia": "Pneumonia", # OBS.: Linda ignora essa classe
           "SARS": "Pneumonia",
           "MERS": "Pneumonia",
           "Streptococcus": "Pneumonia",
           "Klebsiella": "Pneumonia",
           "Chlamydophila": "Pneumonia",
           "Legionella": "Pneumonia",
           "Lung Opacity": "Pneumonia",
           "1": "Pneumonia",
           "std_subst": "Non-COVID"}
# remove_classes = ["todo", "nan", "Unknown"]

filtered_table = filter_table(file_table, mapping, general_case="remove")
table_info(filtered_table)

## SEPARAÇÃO DE TESTES PARA COHEN, FIG1, ACTUALMED E SIRM

In [ ]:
test_patients = {"Pneumonia": ['8', '31'],
                 "COVID-19": ['19', '20', '36', '42', '86', 
                              '94', '97', '117', '132', 
                              '138', '144', '150', '163', '169', '174', '175', '179', '190', '191',
                              'COVID-00024', 'COVID-00025', 'COVID-00026', 'COVID-00027', 'COVID-00029',
                              'COVID-00030', 'COVID-00032', 'COVID-00033', 'COVID-00035', 'COVID-00036',
                              'COVID-00037', 'COVID-00038',
                              'ANON24', 'ANON45', 'ANON126', 'ANON106', 'ANON67',
                              'ANON153', 'ANON135', 'ANON44', 'ANON29', 'ANON201', 
                              'ANON191', 'ANON234', 'ANON110', 'ANON112', 'ANON73', 
                              'ANON220', 'ANON189', 'ANON30', 'ANON53', 'ANON46',
                              'ANON218', 'ANON240', 'ANON100', 'ANON237', 'ANON158',
                              'ANON174', 'ANON19', 'ANON195',
                              'COVID-19(119)', 'COVID-19(87)', 'COVID-19(70)', 'COVID-19(94)', 
                              'COVID-19(215)', 'COVID-19(77)', 'COVID-19(213)', 'COVID-19(81)', 
                              'COVID-19(216)', 'COVID-19(72)', 'COVID-19(106)', 'COVID-19(131)', 
                              'COVID-19(107)', 'COVID-19(116)', 'COVID-19(95)', 'COVID-19(214)', 
                              'COVID-19(129)']}

test_table = []
train_table = []
for row in filtered_table:
    if row["class"] in test_patients and row["id"] in test_patients[row["class"]]:
        test_table.append(row)
    else:
        train_table.append(row)
    
print(f"\nTRAIN:\n-------------------")
table_info(train_table)
print(f"\nTEST:\n-------------------")
table_info(test_table)

## FILTRAGEM DAS CLASSES DE INTERESSE DO RSNA

In [ ]:
rsna_filtered_table = filter_table(rsna.table, mapping, general_case="remove")
table_info(rsna_filtered_table)

## SEPARAÇÃO DE TESTES RSNA

In [ ]:
split = 0.2
rsna_train, rsna_test = split_table(rsna_filtered_table, split)
print(f"\nTRAIN:\n-------------------")
table_info(rsna_train)
print(f"\nTEST:\n-------------------")
table_info(rsna_test)

## JUNTA AS TABLES DE TESTE E TREINO

In [ ]:
from ds_utils import remove_duplicated

train_table += rsna_train
test_table += rsna_test

print(f"\nTRAIN:\n-------------------")
table_info(train_table)
print(f"\nTEST:\n-------------------")
table_info(test_table)

# MONTA O DATASET COPIANDO OS ARQUIVOS DA TABLE (SEPARADOS POR CLASSE)

In [ ]:
from ds_utils import mount_dataset

mounted_dataset_path = "./target_dataset"
if not os.path.isdir(mounted_dataset_path):
    os.mkdir(mounted_dataset_path)
train_path = os.path.join(mounted_dataset_path, "train")
test_path = os.path.join(mounted_dataset_path, "test")

print(f"\nTRAIN:\n-------------------")
mount_dataset(train_path, train_table)
print(f"\nTEST:\n-------------------")
mount_dataset(test_path, test_table)

ModuleNotFoundError: ignored

## TREINAMENTO

In [1]:
mounted_dataset_path = "/content/drive/MyDrive/TG/tg2_COVIDNet/target_dataset"
#mounted_dataset_path = "./target_dataset"
image_size = (240, 240)
total_training_images = 12460
total_test_images = 3080
batch_size = 32
train_max_batches = int(total_training_images/batch_size)
test_max_batches = int(total_test_images/batch_size)
color_mode = "grayscale"
class_mode = "categorical"


In [ ]:
import tensorflow as tf
import os

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

train_datagen = train_datagen.flow_from_directory(
    os.path.join(mounted_dataset_path, "train"),
    shuffle=True,
    target_size=image_size,
    color_mode=color_mode,
    batch_size=batch_size)

test_datagen = test_datagen.flow_from_directory(
    os.path.join(mounted_dataset_path, "test"),
    shuffle=True,
    target_size=image_size,
    color_mode=color_mode,
    batch_size=batch_size)



In [ ]:
import numpy as np

x_train = []
y_train = []

x_test = []
y_test = []

print("Creating training array: ")
for i in range(train_max_batches):
    print(f"  Batch: {i}")
    batch = train_datagen.next()
    x_train.extend(batch[0])
    y_train.extend(batch[1])
x_train = np.array(x_train)
y_train = np.array(y_train)


print("\nCreating test array:")
for i in range(test_max_batches):
    print(f"  Batch: {i}")
    batch = test_datagen.next()
    x_test.extend(batch[0])
    y_test.extend(batch[1])
x_test = np.array(x_test)
y_test = np.array(y_test)


## Save train and test numpy dataset array to disk

*   List item
*   List item



In [ ]:
from numpy import save

save('x_train_data.npy', x_train)
save('y_train_data.npy', y_train)

save('x_test_data.npy', x_test)
save('y_test_data.npy', y_test)


## Load train and test numpy dataset array to disk


In [2]:
import os
from numpy import load

numpy_array_ds_path = '/content/drive/MyDrive/TG/dataset_arrays'

x_train = load(os.path.join(numpy_array_ds_path,'x_train_data.npy'))
y_train = load(os.path.join(numpy_array_ds_path,'y_train_data.npy'))
x_test = load(os.path.join(numpy_array_ds_path,'x_test_data.npy'))
y_test = load(os.path.join(numpy_array_ds_path,'y_test_data.npy'))

In [3]:
print("TRAIN\n----------")
print(x_train.shape)
print(y_train.shape)
print("\nTEST\n----------")
print(x_test.shape)
print(y_test.shape)

TRAIN
----------
(12448, 240, 240, 1)
(12448, 3)

TEST
----------
(3072, 240, 240, 1)
(3072, 3)


In [5]:
import tensorflow as tf

tb_log_dir = "/Users/igorbeduin/Google Drive (beduinigor@gmail.com)/TG/tensorboard"
checkpoint_path = "/home/beduinigor/tg2_COVIDNet/checkpoints/weights.epoch:{epoch:02d}-acc:{accuracy:.2f}.h5"
checkpoint_epochs = 7000

early_stopping = tf.keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    min_delta=0,
                    patience=20,
                    verbose=1,
                    mode="auto",
                    baseline=None,
                    restore_best_weights=True)

tensorboard = tf.keras.callbacks.TensorBoard(
                    log_dir=tb_log_dir,
                    histogram_freq=0,
                    write_graph=True,
                    write_images=False,
                    update_freq="epoch",
                    profile_batch=100000000000,
                    embeddings_freq=0,
                    embeddings_metadata=None)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
                    checkpoint_path,
                    monitor="val_loss",
                    verbose=1,
                    save_best_only=False,
                    save_weights_only=False,
                    mode="auto",
                    save_freq=checkpoint_epochs,
                    options=None)

callbacks = [tensorboard]

### Searching on resnet models

In [7]:
import autokeras as ak

model_path = "/content/drive/MyDrive/TG/models"

input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=False,
    # Do not do data augmentation.
    augment=False,
    )(input_node)
output_node = ak.ClassificationHead()(output_node)

model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    directory=model_path,
    overwrite=False,
    max_trials=20)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    callbacks=callbacks, 
    validation_split=0.2,
    epochs=100,
    verbose=1)

KeyboardInterrupt: ignored

### Searching on every classification model

In [6]:
import autokeras as ak
import tensorflow as tf

model_path = "/content/drive/MyDrive/TG/models"

model = ak.ImageClassifier(
    num_classes=3,
    overwrite=False,
    directory=model_path,
    max_trials=20)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    callbacks=callbacks, 
    validation_split=0.2,
    epochs=100,
    verbose=1)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
image_block_1/b...|vanilla           |?                 
image_block_1/n...|True              |?                 
image_block_1/a...|False             |?                 
image_block_1/c...|3                 |?                 
image_block_1/c...|1                 |?                 
image_block_1/c...|2                 |?                 
image_block_1/c...|True              |?                 
image_block_1/c...|False             |?                 
image_block_1/c...|0.25              |?                 
image_block_1/c...|32                |?                 
image_block_1/c...|64                |?                 
classification_...|flatten           |?                 
classification_...|0.5               |?                 
optimizer         |adam              |?                 
learning_rate     |0.001             |?                 



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 64, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/autokeras/graph.py", line 250, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "/usr/local/lib/python3.6/dist-packages/autokeras/engine/block.py", line 38, in _build_wrapper
    return super()._build_wrapper(hp, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 64, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/autokeras/blocks/wrapper.py", line 89, in build
    if self.normalize is None and hp.Boolean(NORMALIZE):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hyperparamet

Invalid model 0/5
Invalid model 1/5
Invalid model 2/5
Invalid model 3/5
Invalid model 4/5
Invalid model 5/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 64, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/autokeras/graph.py", line 250, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "/usr/local/lib/python3.6/dist-packages/autokeras/engine/block.py", line 38, in _build_wrapper
    return super()._build_wrapper(hp, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 64, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/autokeras/blocks/wrapper.py", line 89, in build
    if self.normalize is None and hp.Boolean(NORMALIZE):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hyperparamet

RuntimeError: ignored

In [ ]:
trained_model = model.export_model()

In [ ]:
trained_model.save("checkpoints/final_model.h5")

## LEITURA DE PESOS

In [ ]:
from tensorflow.keras.models import load_model
import os

checkpoints_path = "/home/beduinigor/tg2_COVIDNet/checkpoints"
auto_model_path = "/media/training/covidnet/auto_model"
best_model_path = "/media/training/covidnet/checkpoints/final_model.h5"

loaded_model = load_model(best_model_path)
print(loaded_model.evaluate(x_test, y_test, batch_size=batch_size))
#for cp in os.listdir(checkpoints_path):
#    loaded_model = load_model(os.path.join(checkpoints_path, cp))
#    print(loaded_model.evaluate(x_test, y_test, batch_size=batch_size))
